<a href="https://colab.research.google.com/github/aveen-d/hello/blob/master/AI21RESCH11003_Assign1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


Method for Stratified Split of entire dataset into train and test

In [ ]:
def stratified_train_test_split(data):
  num_pos = data[:,-1].tolist().count(1)
  num_neg = data[:,-1].tolist().count(0)
  print("Total positive samples: " +str(num_pos))
  print("Total negative samples: "+str(num_neg))
  test_set=[]
  train_set=[]
  p_count=0
  n_count=0
  for i in range(data.shape[0]):
    if data[i,-1] == 1 and p_count <int(num_pos/10):
      test_set.append(data[i])
      p_count=p_count+1
    elif  data[i,-1] == 0 and n_count <int(num_neg/10):
      test_set.append(data[i])
      n_count=n_count+1
    else:
      train_set.append(data[i])
  test_set=np.array(test_set)
  training_set = np.array(train_set)
  return training_set, test_set

Method for Stratified K fold split of train dataset into 10 folds

In [ ]:
def stratified_K_fold(training_set,n_folds):
  pos_data=[]
  neg_data=[]
  for i in range(training_set.shape[0]):
    if training_set[i,-1] == 1:
      pos_data.append(training_set[i])
    else: 
      neg_data.append(training_set[i])
  pos_data=np.array(pos_data)
  neg_data=np.array(neg_data)
  pos_folds = np.array_split(pos_data,10)
  neg_folds = np.array_split(neg_data, 10)
  print("Pos_data in training set: "+str(pos_data.shape))
  print("Neg_data in training set: "+str(neg_data.shape))
  fold_data=[]
  for i in range(n_folds):
    temp = np.concatenate((pos_folds[i],neg_folds[i]),axis=0)
    np.random.shuffle(temp)
    fold_data.append(temp)
  return fold_data



## Univariate Binary Split Decision Tree


Define a class node with the shown attributes


In [ ]:
class Node:
  def __init__(self):
    self.left_child=None
    self.right_child=None
    self.impurity_value=None
    self.feature_value=None
    self.threshold = None
    self.info_gain = None
    self.label=None

Calculate Gini Index

In [ ]:
def gini(data):
  # count number of positive and negative samples
  n_neg = data.tolist().count(0)
  n_pos= data.tolist().count(1)
  
  # calculate probabilities
  p_pos = round(n_pos / data.shape[0], 3)
  p_neg = round(n_neg / data.shape[0], 3)

  if p_pos == 1 or p_neg == 1:
    gini_ = 0
  else:
    gini_ = 1 - (np.square(p_pos)+ np.square(p_neg))
  return gini_

Calculate Entropy value

In [ ]:
def entropy(data):
  # count number of positive and negative samples
  n_neg = data.tolist().count(0)
  n_pos= data.tolist().count(1)

  #calculate probabilities
  p_pos = round(n_pos / data.shape[0], 3)
  p_neg = round(n_neg / data.shape[0], 3)
  
  if p_pos == 1 or p_neg == 1:
    entropy_ = 0
  else:
    entropy_ = - ((p_pos*np.log2(p_pos)) + (p_neg*np.log2(p_neg)))
  return entropy_

Calculate Information gain

In [ ]:
def Info_Gain(left_data, right_data, data):
  n_left = left_data.shape[0]
  n_right = right_data.shape[0]
  total_ = data.shape[0]
  ### use gini index  
  impurity_value = ((n_left/total_) * gini(left_data))+ ((n_right/total_) * gini(right_data))
  info_gain = gini(data) - impurity_value

  ### use entropy
  #impurity_value = ((n_left/total_) * entropy(left_data))+ ((n_right/total_) * entropy(right_data))
  #info_gain = entropy(data) - impurity_value

  return info_gain, impurity_value

Method to find the feature that creates the best split

In [ ]:
def node_split(data):
  max_feature={'threshold':None,'impurity_value':None, 'feature_index':None, 'info_gain': None,'left_data':None, 'right_data':None} # dictionary to track best feature to split
  max_ig = -100000000
  for i in range(data.shape[1]-1):    # Loop over all features
    unique_values = np.unique(data[:,i])
    for j in range(len(unique_values)-2): # Loop over each unique value of each feature
      #mid= (unique_values[j])     # threshold as the unique value
      #mid= (unique_values[j] + unique_values[j+1])/2     # threshold as the midpoint of unique value and unique value+1 (change the for loop condition to range(len(unique_values)-1))
      mid= (unique_values[j] + unique_values[j+1] + unique_values[j+2])/3     # threshold as the average of unique value, unique value+1 and unique value+2 (change the for loop condition to range(len(unique_values)-2))

      # split data into left and right accordingly
      
      temp1=[]
      temp2=[]
      for row in data:
        if row[i]<=mid:
          temp1.append(row)
        elif row[i]>mid:
          temp2.append(row)
      left_data = np.array(temp1)
      right_data = np.array(temp2)

      #calculate Information Gain and check the maximum information gain
      if left_data.shape[0] > 0 and right_data.shape[0] > 0:
        info_gain, impurity_value = Info_Gain(left_data[:,-1], right_data[:,-1], data[:,-1])
        if info_gain > max_ig:
          max_ig = info_gain
          max_feature['threshold'] = mid
          max_feature['impurity_value'] = impurity_value
          max_feature['feature_index'] = i
          max_feature['info_gain'] = info_gain
          max_feature['left_data'] = left_data
          max_feature['right_data'] = right_data
  
  return max_feature['feature_index'], max_feature['left_data'], max_feature['right_data'], max_feature['threshold'], max_feature['impurity_value'], max_feature['info_gain']


Method that creates tree

In [ ]:
def tree(data, current_depth):
  # Parameters to control overfitting of Decision Tree
  max_depth = 5
  min_sample = 10

  if(current_depth <= max_depth and data.shape[0] >= min_sample):
    feature,left_data,right_data, threshold, impurity_value, info_gain = node_split(data) # find the best feature at this node to split
    node=Node()
    node.feature_value = feature
    node.threshold= threshold
    node.impurity_value = impurity_value
    node.info_gain = info_gain
    node.left_child = tree(left_data, current_depth+1)     # Create left node
    node.right_child = tree(right_data, current_depth+1)   # Create right node
    return node

# Treat as Base Case. Assigns a label to leaf node
  node=Node()
  pos = data[:,-1].tolist().count(1)
  neg = data[:,-1].tolist().count(0)
  if pos >= neg:
    node.label= int(1)
  else:
    node.label = int(0)
  
  return node




In [ ]:
# FoML Assign 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.

import csv
import numpy as np

# Enter You Name Here
myname = "Aveen-Dayal" # or "Amar-Akbar-Antony"

# Implement your decision tree below
class DecisionTree():
    tree = {}

    def learn(self, training_dataset):
        # implement this function

        self.tree = tree(training_dataset,0)
        return self.tree

    # implement this function
    def classify(self, test_instance,tree):
        #result = 0 # baseline: always classifies as 0
        if tree.label != None:  
          result = tree.label
          return result
        else:
          instance_val = test_instance[tree.feature_value]
          if instance_val <= tree.threshold:
            return self.classify(test_instance, tree.left_child)
          else:
            return self.classify(test_instance, tree.right_child)

def run_decision_tree():

    # Load data set
    with open("/gdrive/MyDrive/foml datasets/wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print ("Number of records: "+ str(len(data)))

    # Split training/test sets
    # You need to modify the following code for cross validation.

    data=np.array(data).astype(np.float)
    np.random.shuffle(data)
    training_set,test_set = stratified_train_test_split(data) # splits the data into 10% test and 90% train.

    # Split dataset into K folds
    n_folds = 10
    fold_data = stratified_K_fold(training_set, n_folds)

    Acc_train=[]
    Acc_test=[]

    for k in range(n_folds):
      test_dataset = fold_data[k]
      temp=[]
      for j in range(n_folds):
        if j != k:
          temp.append(fold_data[j])
      train_dataset = np.vstack(temp)
      print("pp")
      print(train_dataset.shape)
      np.random.shuffle(train_dataset)
      tree = DecisionTree()
    # Construct a tree using training set
      learnt_tree=tree.learn( train_dataset )

    # Classify the test set using the tree we just constructed
      results_test = []
      for instance in test_dataset:
        result = tree.classify( instance[:-1], learnt_tree )
        results_test.append( result == instance[-1])
      
      results_train = []
      for instance in train_dataset:
        result = tree.classify( instance[:-1], learnt_tree )
        results_train.append( result == instance[-1])


    # Accuracy
      accuracy_train = float(results_train.count(True))/float(len(results_train))
      print (" Train accuracy at fold "+ str(k+1)+ ": " +str(accuracy_train))       
      Acc_train.append(accuracy_train)
    
      accuracy_test = float(results_test.count(True))/float(len(results_test))
      print (" Test accuracy at fold "+ str(k+1)+ ": " +str(accuracy_test))       
      Acc_test.append(accuracy_test)
      print("------END OF FOLD "+str(k+1)+" -----")
    
    return Acc_test, Acc_train
    
    '''
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()
    '''

if __name__ == "__main__":
    Acc_test, Acc_train = run_decision_tree()

Number of records: 4898
Total positive samples: 1060
Total negative samples: 3838
Pos_data in training set: (954, 12)
Neg_data in training set: (3455, 12)
 Train accuracy at fold 1: 0.8437106125535669
 Test accuracy at fold 1: 0.8190045248868778
------END OF FOLD 1 -----
 Train accuracy at fold 2: 0.8391731787244769
 Test accuracy at fold 2: 0.8190045248868778
------END OF FOLD 2 -----
 Train accuracy at fold 3: 0.8553062767834636
 Test accuracy at fold 3: 0.8076923076923077
------END OF FOLD 3 -----
 Train accuracy at fold 4: 0.8459793294681119
 Test accuracy at fold 4: 0.8190045248868778
------END OF FOLD 4 -----
 Train accuracy at fold 5: 0.8432459677419355
 Test accuracy at fold 5: 0.8344671201814059
------END OF FOLD 5 -----
 Train accuracy at fold 6: 0.8384983623078861
 Test accuracy at fold 6: 0.8068181818181818
------END OF FOLD 6 -----
 Train accuracy at fold 7: 0.8480725623582767
 Test accuracy at fold 7: 0.7954545454545454
------END OF FOLD 7 -----
 Train accuracy at fold 8:

In [ ]:
print(" Final Train accuracy after 10 folds: "+ str(np.average(np.array(Acc_train)))+ "+/- "+str(np.std(np.array(Acc_train))))
print(" Final Test accuracy after 10 folds: "+ str(np.average(np.array(Acc_test)))+ " +/- "+str(np.std(np.array(Acc_test))))

 Final Train accuracy after 10 folds: 0.8454172734886066+/- 0.004548409294559074
 Final Test accuracy after 10 folds: 0.8142354820716164 +/- 0.009692837304730398


## Univariate Three-way Split Decision Tree


In [ ]:
class NodeMulti:
  def __init__(self):
    self.left_child=None
    self.right_child=None
    self.middle_child= None
    self.impurity_value=None
    self.feature_value=None
    self.threshold = []
    self.info_gain = None
    self.label=None

Method to calculate Information gain. Uses Gini and Entropy methods from above

In [ ]:
def Info_Gain_multi(left_data, right_data, middle_data, data):

  n_left = left_data.shape[0]
  n_right = right_data.shape[0]
  n_middle = middle_data.shape[0]
  total_ = data.shape[0]

  # Use Gini Index
  #impurity_value = ((n_left/total_) * gini(left_data))+ ((n_right/total_) * gini(right_data)) +((n_middle/total_) * gini(middle_data))
  #info_gain = gini(data) - impurity_value

  # Use Entropy Value
  impurity_value = ((n_left/total_) * entropy(left_data))+ ((n_right/total_) * entropy(right_data)) +((n_middle/total_) * entropy(middle_data))
  info_gain = entropy(data) - impurity_value

  return info_gain, impurity_value

Method to split a node three way

In [ ]:
def node_multisplit(data):
  max_feature={'threshold':[],'impurity_value':None, 'feature_index':None, 'info_gain': None,'left_data':None, 'right_data':None, 'middle_data':None}
  max_ig = -100000000
  for i in range(data.shape[1]-1): # loop over each feature
    unique_values = np.unique(data[:,i])
    for j in range(5,len(unique_values)-5):   # loop over unique value of each feature
      # Splits data into three parts
      left_data = np.array([row for row in data if row[i]<= unique_values[j-5]]) # Change the integer value to have more wider range using which we split the data. Change the value accordingly in the for loop.
      middle_data = np.array([row for row in data if (row[i]< unique_values[j+5] and row[i]> unique_values[j-5])])
      right_data = np.array([row for row in data if (row[i]>=unique_values[j+5])])

      if left_data.shape[0] > 0 and right_data.shape[0] > 0 and middle_data.shape[0] > 0:
        info_gain, impurity_value = Info_Gain_multi(left_data[:,-1], right_data[:,-1], middle_data[:,-1], data[:,-1])
        if info_gain > max_ig:
          max_ig = info_gain
          max_feature['threshold'] = [unique_values[j-5], unique_values[j+5]]
          max_feature['impurity_value'] = impurity_value
          max_feature['feature_index'] = i
          max_feature['info_gain'] = info_gain
          max_feature['left_data'] = left_data
          max_feature['right_data'] = right_data
          max_feature['middle_data'] = middle_data
  
  return max_feature['feature_index'], max_feature['left_data'], max_feature['right_data'], max_feature['middle_data'],max_feature['threshold'], max_feature['impurity_value'], max_feature['info_gain']


Build tree

In [ ]:
def tree_multi(data, current_depth):
  

  max_depth = 3
  min_sample = 30
  
  if(current_depth <= max_depth and data.shape[0] >= min_sample):
    feature,left_data,right_data, middle_data, threshold, impurity_value, info_gain = node_multisplit(data) 
    node=NodeMulti()
    node.feature_value = feature
    node.threshold= threshold
    node.impurity_value = impurity_value
    node.info_gain = info_gain
    node.left_child = tree_multi(left_data, current_depth+1)
    node.middle_child = tree_multi(middle_data, current_depth+1)
    node.right_child = tree_multi(right_data, current_depth+1)
    return node

# Base Case
  node=NodeMulti()
  pos = data[:,-1].tolist().count(1)
  neg = data[:,-1].tolist().count(0)
  if pos >= neg:
    node.label= int(1)
  else:
    node.label = int(0)
  
  return node




In [ ]:
# FoML Assign 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.

import csv
import numpy as np

# Enter You Name Here
myname = "Aveen-Dayal" # or "Amar-Akbar-Antony"

# Implement your decision tree below
class DecisionTree():
    tree = {}

    def learn(self, training_dataset):
        # implement this function

        self.tree = tree_multi(training_dataset,0)
        return self.tree

    # implement this function
    def classify(self, test_instance,tree):
        #result = 0 # baseline: always classifies as 0
        if tree.label != None:  
          result = tree.label
          return result
        else:
          instance_val = test_instance[tree.feature_value]
          if instance_val <= tree.threshold[0]:
            return self.classify(test_instance, tree.left_child)
          elif instance_val > tree.threshold[0] and instance_val < tree.threshold[1]:
            return self.classify(test_instance, tree.middle_child)
          else:
            return self.classify(test_instance, tree.right_child)

def run_decision_tree():

    # Load data set
    with open("/gdrive/MyDrive/foml datasets/wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print ("Number of records: "+ str(len(data)))

    # Split training/test sets
    # You need to modify the following code for cross validation.

    data=np.array(data).astype(np.float)
    np.random.shuffle(data)
    training_set,test_set = stratified_train_test_split(data)


    n_folds = 10
    fold_data = stratified_K_fold(training_set, n_folds)

    Acc_train=[]
    Acc_test=[]

    for k in range(n_folds):
      test_dataset = fold_data[k]
      temp=[]
      for j in range(n_folds):
        if j != k:
          temp.append(fold_data[j])
      train_dataset = np.vstack(temp)
      np.random.shuffle(train_dataset)
      tree = DecisionTree()
    # Construct a tree using training set
      learnt_tree=tree.learn( train_dataset )

    # Classify the test set using the tree we just constructed
      results_test = []
      for instance in test_dataset:
        result = tree.classify( instance[:-1], learnt_tree )
        results_test.append( result == instance[-1])
      
      results_train = []
      for instance in train_dataset:
        result = tree.classify( instance[:-1], learnt_tree )
        results_train.append( result == instance[-1])


    # Accuracy
      accuracy_train = float(results_train.count(True))/float(len(results_train))
      print (" Train accuracy at fold "+ str(k+1)+ ": " +str(accuracy_train))       
      Acc_train.append(accuracy_train)

      accuracy_test = float(results_test.count(True))/float(len(results_test))
      print (" Test accuracy at fold "+ str(k+1)+ ": " +str(accuracy_test))       
      Acc_test.append(accuracy_test)
      print("------END OF FOLD "+str(k+1)+" -----")
    
    return Acc_test, Acc_train
    
    '''
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()
    '''

if __name__ == "__main__":
    Acc_test, Acc_train = run_decision_tree()


In [ ]:
print(" Final Train accuracy after 10 folds: "+ str(np.average(np.array(Acc_train)))+ "+/- "+str(np.std(np.array(Acc_train))))
print(" Final Test accuracy after 10 folds: "+ str(np.average(np.array(Acc_test)))+ " +/- "+str(np.std(np.array(Acc_test))))

## Multi Variate Binary Split Decision Tree

In [ ]:
import pandas as pd
df=pd.read_csv("/gdrive/MyDrive/foml datasets/wine-dataset.csv")
df.head()

Find correlation between features

In [ ]:
df.corr()

As explained in the report we select top 4 least correlated features for each feature and use that for findinf best split.

In [ ]:
feature_corr={'0':[1,4,9,10],'1':[0,7,8,9], '2':[1,4,6,7],'3':[1,4,8,9],'4':[0,1,8,9],'5':[0,1,8,9],'6':[0,1,8,9],'7':[1,2,8,9],'8':[1,4,5,6,],'9':[0,3,4,10],'10':[1,2,8,9]}

Node class for mutlivariate

In [ ]:
class NodeMultivar:
  def __init__(self):
    self.left_child=None
    self.right_child=None
    self.impurity_value=None
    self.feature_value=[]
    self.threshold = []
    self.info_gain = None
    self.label=None

Method to split node using 2 features.

In [ ]:
def multi_node_split(data):
  max_feature={'threshold':None,'impurity_value':None, 'feature_index':None, 'info_gain': None,'left_data':None, 'right_data':None}
  max_ig = -100000000
  for i in range(data.shape[1]-1):
    for k in feature_corr[str(i)][:2]: 
      unique_values1 = np.unique(data[:,i])
      unique_values2 = np.unique(data[:,k])
      for m in range(len(unique_values1)):
        for j in range(len(unique_values2)):
          mid1= (unique_values1[m])
          mid2 = (unique_values2[j])
          temp1=[]
          temp2=[]
          for row in data:
            if row[i]==mid1 and row[k]==mid2:
              temp1.append(row)
            elif row[i]!=mid1 or row[k]!=mid2:
            temp2.append(row)
          left_data = np.array(temp1)
          right_data = np.array(temp2)
          if left_data.shape[0] > 0 and right_data.shape[0] > 0:
            info_gain, impurity_value = Info_Gain(left_data[:,-1], right_data[:,-1], data[:,-1])
            if info_gain > max_ig:
              max_ig = info_gain
              max_feature['threshold'] = [mid1, mid2]
              max_feature['impurity_value'] = impurity_value
              max_feature['feature_index'] = [i,k]
              max_feature['info_gain'] = info_gain
              max_feature['left_data'] = left_data
              max_feature['right_data'] = right_data

  return max_feature['feature_index'], max_feature['left_data'], max_feature['right_data'], max_feature['threshold'], max_feature['impurity_value'], max_feature['info_gain']


Build Multi variate tree

In [ ]:
def tree_multi_var(data, current_depth):
  

  max_depth = 4
  min_sample = 30

  if(current_depth <= max_depth and data.shape[0] >= min_sample):
    feature,left_data,right_data, threshold, impurity_value, info_gain = multi_node_split(data)
    node=NodeMultivar()
    node.feature_value = feature
    node.threshold= threshold
    node.impurity_value = impurity_value
    node.info_gain = info_gain
    node.left_child = tree_multi_var(left_data, current_depth+1)
    node.right_child = tree_multi_var(right_data, current_depth+1)
    return node

# Base Case
  node=NodeMultivar()
  pos = data[:,-1].tolist().count(1)
  neg = data[:,-1].tolist().count(0)
  if pos >= neg:
    node.label= int(1)
  else:
    node.label = int(0)
  
  return node




In [ ]:
# FoML Assign 1 Code Skeleton
# Please use this outline to implement your decision tree. You can add any code around this.

import csv
import numpy as np

# Enter You Name Here
myname = "Aveen-Dayal" # or "Amar-Akbar-Antony"

# Implement your decision tree below
class DecisionTree():
    tree = {}

    def learn(self, training_dataset):
        # implement this function

        self.tree = tree_multi_var(training_dataset,0)
        return self.tree

    # implement this function
    def classify(self, test_instance,tree):
        #result = 0 # baseline: always classifies as 0
        if tree.label != None:  
          result = tree.label
          return result
        else:
          instance_val1 = test_instance[tree.feature_value[0]]
          instance_val2 = test_instance[tree.feature_value[1]]
                    
          if instance_val1 == tree.threshold[0] and instance_val2 == tree.threshold[1]:
            return self.classify(test_instance, tree.left_child)
          elif instance_val1 != tree.threshold[0] or instance_val2 != tree.threshold[1]:
            return self.classify(test_instance, tree.right_child)
          

def run_decision_tree():

    # Load data set
    with open("/gdrive/MyDrive/foml datasets/wine-dataset.csv") as f:
        next(f, None)
        data = [tuple(line) for line in csv.reader(f, delimiter=",")]
    print ("Number of records: "+ str(len(data)))

    # Split training/test sets
    # You need to modify the following code for cross validation.

    data=np.array(data).astype(np.float)
    np.random.shuffle(data)
    training_set,test_set = stratified_train_test_split(data)

    n_folds = 10
    fold_data = stratified_K_fold(training_set, n_folds)

    Acc_test=[]
    Acc_train=[]

    for k in range(n_folds):
      test_dataset = fold_data[k]
      temp=[]
      for j in range(n_folds):
        if j != k:
          temp.append(fold_data[j])
      train_dataset = np.vstack(temp)
      np.random.shuffle(train_dataset)
      tree = DecisionTree()
    # Construct a tree using training set
      learnt_tree=tree.learn( train_dataset )

    # Classify the test set using the tree we just constructed
      results_test = []
      for instance in test_dataset:
        result = tree.classify( instance[:-1], learnt_tree )
        results_test.append( result == instance[-1])
      
      results_train = []
      for instance in train_dataset:
        result = tree.classify( instance[:-1], learnt_tree )
        results_train.append( result == instance[-1])


    # Accuracy
      accuracy_train = float(results_train.count(True))/float(len(results_train))
      print (" Train accuracy at fold "+ str(k+1)+ ": " +str(accuracy_train))       
      Acc_train.append(accuracy_train)

      accuracy_test = float(results_test.count(True))/float(len(results_test))
      print (" Test accuracy at fold "+ str(k+1)+ ": " +str(accuracy_test))       
      Acc_test.append(accuracy_test)
      print("------END OF FOLD "+str(k+1)+" -----")
    
    return Acc_test, Acc_train
    
    '''
    # Writing results to a file (DO NOT CHANGE)
    f = open(myname+"result.txt", "w")
    f.write("accuracy: %.4f" % accuracy)
    f.close()
    '''

if __name__ == "__main__":
    Acc_test, Acc_train = run_decision_tree()
